In [2]:
import pandas as pd
import keras
import numpy as np
import pickle as pk
from io import StringIO
from scipy import spatial
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelBinarizer
from keras.models import Model, Sequential
from keras.layers import LSTM,Dense,Input,Bidirectional, Embedding, Dropout
from keras import regularizers
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy import spatial
from random import shuffle
import pickle as pk
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import one_hot
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

Using TensorFlow backend.


In [3]:
fp = pd.read_csv('fp.csv')

In [4]:
bc = pd.read_csv('bc.csv')

In [5]:
ch = pd.read_csv('chr.csv')

In [6]:
all_of_them = pd.concat([fp, bc, ch])
all_of_them

,Unnamed: 0,text,polarity,factuality
0,0,I have been a coffee drinker on and off since ...,NEUTRAL,EXPERIENCE
1,1,"I notice now that I am older, and perhaps I ha...",NEGATIVE,OPINION
2,2,"When I drink it, if I drink about 2 cups a day...",NEGATIVE,EXPERIENCE
3,3,I always kind of said oh it's just allergies f...,NEGATIVE,OPINION
4,4,"but in fact when I do not consume coffee, I am...",POSITIVE,EXPERIENCE
5,5,I tried to find some information on this but i...,NEGATIVE,EXPERIENCE
6,6,Would be great if anyone had any feedback on t...,POSITIVE,OPINION
7,7,Thanks so much.,POSITIVE,NOT_LABELED
8,8,Are you drinking decaf or caffeinated?,NEUTRAL,FACT
9,9,That does sound like you have a food sensitivi...,NEGATIVE,OPINION


# Polarity

In [6]:
neg_doc = ' '.join(all_of_them[all_of_them['polarity'] == 'NEGATIVE']['text'])
pos_doc = ' '.join(all_of_them[all_of_them['polarity'] == 'POSITIVE']['text'])
neu_doc = ' '.join(all_of_them[all_of_them['polarity'] == 'NEUTRAL']['text'])

In [7]:
len(neu_doc)

146089

In [8]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [10]:
tfidf_pol = TfidfVectorizer(sublinear_tf=True, stop_words='english')

In [11]:
tfidf_pol.fit([neg_doc,pos_doc,neu_doc])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=200, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
X_pol = list(all_of_them[all_of_them['polarity']!='NOT_LABELED']['text'])

In [13]:
X_pol

['I have been a coffee drinker on and off since I was little.',
 'I notice now that I am older, and perhaps I have always had this issue just did not correlate the connection between the two, but I have to wonder if it is possible for me to have an allergy to coffee.',
 'When I drink it, if I drink about 2 cups a day spaced out, it takes about a week for me to get these symptoms: - sore throat, increased nasal drainage and nasal drip in the back of my throat, sometimes slight eye swelling, slight cough.',
 "I always kind of said oh it's just allergies from pollen or I am catching a cold...",
 'but in fact when I do not consume coffee, I am back to normal in about 3 days.',
 'I tried to find some information on this but it seems hard to read up on.',
 'Would be great if anyone had any feedback on this.',
 'Thanks so much.',
 'Are you drinking decaf or caffeinated?',
 'That does sound like you have a food sensitivity to coffee.',
 "I don't think it's the caffeine, but one way you can fig

In [14]:
Y_pol = list(all_of_them[all_of_them['polarity']!='NOT_LABELED']['polarity'])
Y_pol

['NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'POSITIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 

In [15]:
X_train_pol, X_test_pol, Y_train_pol, Y_test_pol = train_test_split(X_pol, Y_pol, test_size = 0.2)

In [16]:
from sklearn.preprocessing import LabelEncoder
le_pol = LabelEncoder()
le_pol.fit(Y_pol)

Y_train_pol = le_pol.transform(Y_train_pol)
Y_test_pol = le_pol.transform(Y_test_pol)

# Guassian NB

In [116]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(tfidf_pol.transform(X_train_pol).toarray(),Y_train_pol)

GaussianNB(priors=None)

In [117]:
from sklearn.metrics import classification_report

y_pred = gnb.predict(tfidf_pol.transform(X_test_pol).toarray())
print(classification_report(Y_test_pol, y_pred))

             precision    recall  f1-score   support

          0       0.58      0.61      0.59       227
          1       0.70      0.57      0.63       329
          2       0.45      0.62      0.52       134

avg / total       0.61      0.59      0.60       690



# Random Forest

In [118]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(tfidf_pol.transform(X_train_pol).toarray(),Y_train_pol)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [119]:
from sklearn.metrics import classification_report

y_pred = rf.predict(tfidf_pol.transform(X_test_pol).toarray())
print(classification_report(Y_test_pol, y_pred))

             precision    recall  f1-score   support

          0       0.54      0.47      0.50       227
          1       0.59      0.75      0.66       329
          2       0.74      0.40      0.52       134

avg / total       0.60      0.59      0.58       690



# Passive Agressive Classifier

In [120]:
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier()
pac.fit(tfidf_pol.transform(X_train_pol).toarray(),Y_train_pol)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
              fit_intercept=True, loss='hinge', max_iter=None, n_iter=None,
              n_jobs=1, random_state=None, shuffle=True, tol=None,
              verbose=0, warm_start=False)

In [121]:
from sklearn.metrics import classification_report

y_pred = pac.predict(tfidf_pol.transform(X_test_pol).toarray())
print(classification_report(Y_test_pol, y_pred))

             precision    recall  f1-score   support

          0       0.65      0.49      0.56       227
          1       0.62      0.84      0.71       329
          2       0.82      0.43      0.57       134

avg / total       0.67      0.65      0.63       690



# SVM

In [122]:
from sklearn import svm

svm_clf = svm.SVC()
svm_clf.fit(tfidf_pol.transform(X_train_pol).toarray(),Y_train_pol)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [123]:
from sklearn.metrics import classification_report

y_pred = svm_clf.predict(tfidf_pol.transform(X_test_pol).toarray())
print(classification_report(Y_test_pol, y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       227
          1       0.48      1.00      0.65       329
          2       0.00      0.00      0.00       134

avg / total       0.23      0.48      0.31       690



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [124]:
from sklearn.metrics import f1_score

y_pred = svm_clf.predict(tfidf_pol.transform(X_test_pol).toarray())
print(f1_score(Y_test_pol, y_pred, average = 'micro'))

0.47681159420289854


![](https://miro.medium.com/max/2000/1*AsCqMAkwy9m0eWmSqITolg.png)

In [126]:
from sklearn import svm

svm_cl = svm.SVC(C=100)
svm_cl.fit(tfidf_pol.transform(X_train_pol).toarray(),Y_train_pol)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [127]:
from sklearn.metrics import classification_report

y_pred = svm_cl.predict(tfidf_pol.transform(X_test_pol).toarray())
print(classification_report(Y_test_pol, y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       227
          1       0.48      1.00      0.65       329
          2       0.00      0.00      0.00       134

avg / total       0.23      0.48      0.31       690



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# factuality

In [7]:
fact_doc = ' '.join(all_of_them[all_of_them['factuality'] == 'FACT']['text'])
exp_doc = ' '.join(all_of_them[all_of_them['factuality'] == 'EXPERIENCE']['text'])
op_doc = ' '.join(all_of_them[all_of_them['factuality'] == 'OPINION']['text'])

In [10]:
op_doc

'I notice now that I am older, and perhaps I have always had this issue just did not correlate the connection between the two, but I have to wonder if it is possible for me to have an allergy to coffee. I always kind of said oh it\'s just allergies from pollen or I am catching a cold... Would be great if anyone had any feedback on this. That does sound like you have a food sensitivity to coffee. I don\'t think it\'s the caffeine, but one way you can figure out if it\'s caffeine is to try black or green tea (both are from the same plant). I know it seems like there shouldn\'t be a risk, since they are steaming, then drying to keep the leaves green. So, I\'m not sure where the lead comes from. But, black tea doesn\'t carry the same risk. Hardest food allergy/intolerance to break when you adore the smells of coffee :( But at the same time frustrating if I have really consumed these things for over 20 some years not knowing it was causing these symptoms and never putting the two together. 

In [41]:
tfidf_fa = TfidfVectorizer(sublinear_tf=True, stop_words='english', ngram_range=(1,2), max_features=5000)

In [42]:
tfidf_fa.fit([fact_doc,exp_doc,op_doc])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
X_fa = list(all_of_them[all_of_them['factuality']!='NOT_LABELED']['text'])

In [17]:
Y_fa = list(all_of_them[all_of_them['factuality']!='NOT_LABELED']['factuality'])
Y_fa

['EXPERIENCE',
 'OPINION',
 'EXPERIENCE',
 'OPINION',
 'EXPERIENCE',
 'EXPERIENCE',
 'OPINION',
 'FACT',
 'OPINION',
 'OPINION',
 'FACT',
 'FACT',
 'FACT',
 'OPINION',
 'OPINION',
 'OPINION',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'FACT',
 'OPINION',
 'OPINION',
 'EXPERIENCE',
 'FACT',
 'OPINION',
 'OPINION',
 'FACT',
 'EXPERIENCE',
 'FACT',
 'OPINION',
 'OPINION',
 'OPINION',
 'FACT',
 'EXPERIENCE',
 'FACT',
 'EXPERIENCE',
 'OPINION',
 'OPINION',
 'FACT',
 'OPINION',
 'EXPERIENCE',
 'OPINION',
 'EXPERIENCE',
 'FACT',
 'OPINION',
 'EXPERIENCE',
 'EXPERIENCE',
 'OPINION',
 'OPINION',
 'EXPERIENCE',
 'OPINION',
 'OPINION',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'OPINION',
 'OPINION',
 'FACT',
 'EXPERIENCE',
 'FACT',
 'FACT',
 'FACT',
 'OPINION',
 'OPINION',
 'FACT',
 'OPINION',
 'EXPERIENCE',
 'EXPERIENCE',
 'EXPERIENCE',
 'FACT',
 'FACT',
 'FACT',
 'OPINION',
 'OPINION',
 'FACT',
 'OPINION',
 'OPINION',
 'EXPERIEN

In [18]:
X_train_fa, X_test_fa, Y_train_fa, Y_test_fa = train_test_split(X_fa, Y_fa, test_size = 0.2)

In [19]:
from sklearn.preprocessing import LabelEncoder
le_fa = LabelEncoder()
le_fa.fit(Y_fa)

Y_train_fa = le_fa.transform(Y_train_fa)
Y_test_fa = le_fa.transform(Y_test_fa)

# Gaussian NB

In [43]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(tfidf_fa.transform(X_train_fa).toarray(),Y_train_fa)

from sklearn.metrics import classification_report

y_pred = gnb.predict(tfidf_fa.transform(X_test_fa).toarray())
print(classification_report(Y_test_fa, y_pred))

             precision    recall  f1-score   support

          0       0.72      0.70      0.71       309
          1       0.48      0.58      0.52       163
          2       0.63      0.56      0.59       187

avg / total       0.64      0.63      0.63       659



# Passive Aggressive Classifier

In [44]:
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier()
pac.fit(tfidf_fa.transform(X_train_fa).toarray(),Y_train_fa)

from sklearn.metrics import classification_report

y_pred = pac.predict(tfidf_fa.transform(X_test_fa).toarray())
print(classification_report(Y_test_fa, y_pred))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


             precision    recall  f1-score   support

          0       0.73      0.76      0.75       309
          1       0.53      0.42      0.47       163
          2       0.59      0.65      0.62       187

avg / total       0.64      0.65      0.64       659



# Random Forest

In [45]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(tfidf_fa.transform(X_train_fa).toarray(),Y_train_fa)

from sklearn.metrics import classification_report

y_pred = rf.predict(tfidf_fa.transform(X_test_fa).toarray())
print(classification_report(Y_test_fa, y_pred))

             precision    recall  f1-score   support

          0       0.69      0.76      0.72       309
          1       0.53      0.38      0.44       163
          2       0.54      0.57      0.55       187

avg / total       0.60      0.61      0.60       659

